# Bisulfite mapping with Bismark

E4.0
- SRR3083926_1
- SRR3083926_2

E5.5
- SRR3083929_1
- SRR3083929_2

## Commands

- bismark_genome_preparation ~/qbb2020-answers/lab-week6/


- bismark --genome genome/ -1 methylation_fastq/SRR3083926_1.chr6.fastq -2 methylation_fastq/SRR3083926_2.chr6.fastq
- bismark --genome genome/ -1 methylation_fastq/SRR3083929_1.chr6.fastq -2 methylation_fastq/SRR3083929_2.chr6.fastq


- samtools sort SRR3083929_1.chr6_bismark_bt2_pe.bam -o SRR3083929.sorted.bam
- samtools sort SRR3083926_1.chr6_bismark_bt2_pe.bam -o SRR3083926.sorted.bam


- samtools index SRR3083926.sorted.bam
- samtools index SRR3083929.sorted.bam


- bismark_methylation_extractor --bedgraph --comprehensive SRR3083929_1.chr6_bismark_bt2_pe.bam
- bismark_methylation_extractor --bedgraph --comprehensive SRR3083926_1.chr6_bismark_bt2_pe.bam

## IGV image of bedgraph file

<img src="igv.png" />

## Python Script for methylation

### For each gene, find the fold change in mean methylation signal from E4 to E5.5 cells

In [57]:
#Open the files
SRR26_file = open('SRR3083926_1.chr6_bismark_bt2_pe.bedGraph', 'r')
SRR29_file = open('SRR3083929_1.chr6_bismark_bt2_pe.bedGraph', 'r')


SRR26 = SRR26_file.readlines() #reads through each line and makes it a string
SRR29 = SRR29_file.readlines() #essentially allows you to read and parse the file lines

e4 = SRR26[1:] #remove header
e5 = SRR29[1:]

SRR26_file.close()
SRR29_file.close()

In [58]:
#opening reference
f = open('mm10_refseq_genes_chr6_50M_60M.bed', 'r')
refgenes = f.readlines()
#refgenes 

In [49]:
#dictionary of the gene names and coordinates
#ideal dictionary output -> {genename: start, end}
genes = {}

for l in refgenes:
    gname = l.split()[12] #splits the string and selects the gene name
    gene_coord = list(map(int,l.split()[4:6])) 
    #make sure the coordinates are integers
    genes[gname] = gene_coord #adds to the dictionary

genes

{'Mpp6': [50110240, 50198598],
 'Dfna5': [50207402, 50261769],
 'Osbpl3': [50293326, 50456170],
 'Cycs': [50562562, 50566474],
 '5430402O13Rik': [50566642, 50594865],
 'Mir6371': [50572397, 50572508],
 '4921507P07Rik': [50573303, 50596590],
 'Npvf': [50650870, 50654393],
 'C530044C16Rik': [50776114, 50814894],
 'Mir148a': [51269811, 51269910],
 'Gm6559': [51379709, 51392791],
 'Nfe2l3': [51432669, 51458768],
 'Hnrnpa2b1': [51460434, 51469894],
 'Cbx3': [51470615, 51483704],
 'Snx10': [51544522, 51590670],
 'Skap2': [51859164, 52012549],
 'Halr1': [52102948, 52113684],
 'Hoxa1': [52155366, 52158317],
 'Hotairm1': [52158523, 52162020],
 'Hoxa2': [52162416, 52164831],
 'Hoxaas2': [52165673, 52169451],
 'Hoxa3': [52169061, 52213067],
 'Hoxa4': [52189686, 52191703],
 'Hoxaas3': [52201123, 52213597],
 'Hoxa5': [52201753, 52204587],
 'Hoxa6': [52206364, 52208624],
 'Mira': [52214491, 52215288],
 'Hoxa7': [52215623, 52218573],
 'Hoxa9': [52223096, 52227370],
 'Mir196b': [52230080, 52230165],
 

In [55]:
#dictionary to have the ave methyl of each gene in both stages
#ideal output -> {genename : e4 methyl, e5 methyl}

#MAIN DICTIONARY

gene_methyl = {}


for key in genes: #itterate through each key:value pair
    
    
    #E4 METHYLATION CALCULATION 
    
    e4_methyl = 0 #used to add up methylations

    for l in e4:
        e4m = l.split() #split the 
        if int(e4m[1]) >= genes[key][0] and int(e4m[2]) <= genes[key][1]:
        #looking at methyl percentages in gene coordinates
            e4_methyl += float(e4m[3]) 
            #add methyls inbetween positions
            
    ave4m = float(e4_methyl)/(genes[key][1]-genes[key][0])
    #calculation for average methylations in gene
    
    
    #E4 METHYLATION CALCULATION
    
    e5_methyl = 0
    
    for line in e5:
        e5m = line.split()
        if int(e5m[1]) >= genes[key][0] and int(e5m[2]) <= genes[key][1]:
            e5_methyl += float(e5m[3])
    
    ave5m = float(e5_methyl)/(genes[key][1]-genes[key][0])
    
    
    #ADDING NON-ZERO METHYLATIONS TO MAIN DICTIONARY
    
    if ave4m == 0:
        continue #skips zero methylations
    else:
        gene_methyl[key] = [ave4m, ave5m]

gene_methyl

{'Mpp6': [0.12790134133136466, 0.5034217482043373],
 'Dfna5': [0.08020685453687092, 0.4988933237687078],
 'Osbpl3': [0.1129822792353545, 0.6786307178486999],
 'Cycs': [0.07604160211522176, 0.6773014250008116],
 '5430402O13Rik': [0.11587700901381236, 0.5315362512506684],
 '4921507P07Rik': [0.10553609514403091, 0.5063575221818019],
 'Npvf': [0.11353959693443089, 0.7684197721098089],
 'C530044C16Rik': [0.18346039955116797, 0.5358739163535451],
 'Mir148a': [1.0101010101010102, 0.0],
 'Gm6559': [0.1576017452794704, 0.7265708607246598],
 'Nfe2l3': [0.12674805770326675, 0.5715280096482442],
 'Hnrnpa2b1': [0.04651162790697675, 0.38704318936877075],
 'Cbx3': [0.0964765364261124, 0.5655179751950443],
 'Snx10': [0.13686813028062925, 0.9595956396268208],
 'Skap2': [0.12350655559737278, 0.5069987063459374],
 'Halr1': [0.018628912071535022, 0.7022878078205947],
 'Hotairm1': [0.028595939376608523, 0.057191878753217046],
 'Hoxa2': [0.00690131124913735, 0.10489993098688753],
 'Hoxaas2': [0.056719352643

In [56]:
#dictionary for the genes and the methylation fold change between e4 and e5.5

#ideal dictionary output -> {genename: foldchange}

fold_chg = {}

for key in gene_methyl: #itterate through previous dictionary
    
    chg = (gene_methyl[key][1] - gene_methyl[key][0])/gene_methyl[key][0]
    #formula for fold change (e5-e4)/e4
    
    fold_chg[key] = chg #set dictionary key:value pair
    
fold_chg

{'Mpp6': 2.9360161743736573,
 'Dfna5': 5.22008339124392,
 'Osbpl3': 5.006523522463532,
 'Cycs': 7.906985204947854,
 '5430402O13Rik': 3.5870725847550147,
 '4921507P07Rik': 3.7979558225150165,
 'Npvf': 5.767857142857142,
 'C530044C16Rik': 1.9209241758142324,
 'Mir148a': -1.0,
 'Gm6559': 3.6101701439679728,
 'Nfe2l3': 3.509165820799114,
 'Hnrnpa2b1': 7.321428571428569,
 'Cbx3': 4.861715150068145,
 'Snx10': 6.01109628413351,
 'Skap2': 3.105034780491621,
 'Halr1': 36.69880952380952,
 'Hotairm1': 1.0,
 'Hoxa2': 14.199999999999973,
 'Hoxaas2': 0.6799999999999999,
 'Hoxa3': 2.217967227315383,
 'Hoxa4': 14.50000000000002,
 'Hoxaas3': 1.2883116883116883,
 'Hoxa5': 4.80952380952381,
 'Hoxa6': 16.999999999999964,
 'Hoxa7': 1.0416666666666667,
 'Hoxa9': 3.439130434782609,
 'Mir196b': -1.0,
 'Hoxa10': -0.16000000000000011,
 'Hoxa11': 0.0,
 'Hoxa11os': 4.666666666666667,
 'Hoxa13': 1.0,
 'Hottip': 1.5739130434782609,
 'Evx1os': 1.2966507177033497,
 'Evx1': 0.04545454545454538,
 '1700094M24Rik': 5.862